In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")


from sae.train import ModelTrainer
from sae.config import create_config, Config
from sae.utils import get_blog_checkpoint, get_blog_sparsity, create_lineplot_histogram

from unlearning.metrics import calculate_wmdp_bio_metrics_hf, get_loss_added_rmu_model
from unlearning.tool import get_basic_gemma_2b_it_layer9_act_store
from unlearning.var import gemma_2b_it_rmu_model_names
from unlearning.metrics import all_permutations
from unlearning.metrics import calculate_metrics_rmu

from transformer_lens import HookedTransformer, utils
from sae.metrics import compute_metrics_post_by_text



import plotly.express as px
import plotly.graph_objs as go
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformer_lens import HookedTransformer
import time

import torch

torch.set_grad_enabled(False)

In [2]:
print("Getting Hugging Face model")

hf_model_name = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto')

transformer_lens_model_name = "google/gemma-2b-it"
base_model = HookedTransformer.from_pretrained(transformer_lens_model_name, hf_model=hf_model, tokenizer=tokenizer)

print("done")

Getting Hugging Face model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer
done


In [3]:
activation_store = get_basic_gemma_2b_it_layer9_act_store(base_model)

buffer
dataloader


In [4]:
hf_model_name = 'eoinf/gemma_2b_it_rmu_s60_a1000'

hf_model = AutoModelForCausalLM.from_pretrained(hf_model_name,
                                                torch_dtype='auto')

rmu_model = HookedTransformer.from_pretrained(transformer_lens_model_name,
                                              hf_model=hf_model,
                                              tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


In [8]:
unlearning_dataset = ['wmdp-bio']
side_effect_dataset_names =  ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']

dataset_names = unlearning_dataset + side_effect_dataset_names


In [6]:
metrics = calculate_metrics_rmu(rmu_model,
                     dataset_names=dataset_names,
                     metric_params={'wmdp-bio': {'target_metric': 'correct'}},
                     get_baseline_metrics=False,
                     compute_loss=False,
                     n_batch_loss_added=2,
                     activation_store=None,
                     )

100%|██████████| 3/3 [00:00<00:00,  5.96it/s]


In [11]:
metrics['college_biology']['total_correct']

14

In [13]:
14/15

0.9333333333333333

In [7]:
[metrics[x]['mean_correct'] for x in dataset_names]

[0.39534884691238403, 1.0, 1.0, 1.0, 1.0, 0.9333333969116211]

In [8]:
metrics_list = []
loss_added_list = []

# model_names = ['gemma_2b_it_rmu_6', 'gemma_2b_it_rmu_10', 'gemma_2b_it_rmu_30', 'gemma_2b_it_rmu_60', 'gemma_2b_it_rmu_100']
model_names = ['eoinf/gemma_2b_it_rmu_s60_a1000']

unlearning_dataset = ['wmdp-bio']
side_effect_dataset_names =  ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']

dataset_names = unlearning_dataset + side_effect_dataset_names

for hf_model_name in model_names:


    metrics = calculate_metrics_rmu(rmu_model,
                         dataset_names=dataset_names,
                         metric_params={'wmdp-bio': {'target_metric': 'correct'}},
                         get_baseline_metrics=False,
                         compute_loss=False,
                         n_batch_loss_added=2,
                         activation_store=None,
                         )
    
    metrics_list.append(metrics)
    
    # print("done metrics")
    
    # loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=20)
    
    # loss_added_list.append(loss_added)
    # print("done", hf_model_name)
    
print("done")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 

In [7]:
dataset_names = ['wmdp-bio']
[metrics_list[0][x]['mean_correct'] for x in dataset_names]

[0.39534884691238403]

In [ ]:
calculate_MCQ_metrics(
    model, 
    dataset_name='wmdp-bio',
    target_metric=None,
    question_subset=None, 
    question_subset_file=None, 
    permutations=[[0, 1, 2, 3]], 
    verbose=True, 
    without_question=False,
    **kwargs
)